In [1]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main'
!pip install -r requirements.txt
!pip install wandb albumentations scikit-learn

/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main
  Cloning https://github.com/pyvoxel/pyvoxel to /tmp/pip-req-build-1zvnrug9
  Running command git clone --filter=blob:none --quiet https://github.com/pyvoxel/pyvoxel /tmp/pip-req-build-1zvnrug9
  Resolved https://github.com/pyvoxel/pyvoxel to commit bacd0f283f4bacb6a3de9d7fd05062788f1fbc15
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/Lightning-AI/metrics.git (to revision release/stable) to /tmp/pip-req-build-cghhfke8
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/metrics.git /tmp/pip-req-build-cghhfke8
  Running command git checkout -b release/stable --track origin/release/stable
  Switched to a new branch 'release/stable'
  Branch 'release/stable' set up to track remote branch 'release/stable' from 'origin'.
  Resolved https://github.com/Lightning-AI/metrics.gi

In [3]:
# Создаёт папку /content/data/images, если её ещё нет.
!mkdir -p /content/data/images

In [ ]:
# !rm -rf /content/data/images/*

In [4]:
# Распаковка фото
!tar -xvzf '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/data/images/images_001.tar.gz' -C /content/data/images --strip-components=1
!tar -xvzf '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/data/images/images_002.tar.gz' -C /content/data/images --strip-components=1
!tar -xvzf '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/data/images/images_003.tar.gz' -C /content/data/images --strip-components=1
!tar -xvzf '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/data/images/images_004.tar.gz' -C /content/data/images --strip-components=1

Выходные данные были обрезаны до нескольких последних строк (5000).
images/00007632_002.png
images/00008864_003.png
images/00008692_000.png
images/00006850_027.png
images/00007359_000.png
images/00006642_033.png
images/00007698_000.png
images/00006981_001.png
images/00008841_047.png
images/00008856_000.png
images/00007712_000.png
images/00008051_024.png
images/00006806_001.png
images/00007093_015.png
images/00007795_005.png
images/00006718_000.png
images/00008266_000.png
images/00007122_000.png
images/00007109_034.png
images/00008888_029.png
images/00007034_068.png
images/00008489_016.png
images/00008522_064.png
images/00008745_011.png
images/00006769_013.png
images/00007787_002.png
images/00007131_004.png
images/00008535_008.png
images/00008363_005.png
images/00008191_002.png
images/00008518_001.png
images/00007713_001.png
images/00007285_003.png
images/00009229_019.png
images/00007676_013.png
images/00007867_000.png
images/00007810_006.png
images/00008008_031.png
images/00007277_000.

In [5]:
# Копирует CSV-файл с метками (таблицей) из Google Диска в рабочую папку Colab
!cp '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/data/Data_Entry_2017_v2020.csv' /content/data/

In [6]:
# Создаёт символическую ссылку (ярлык) на папку с изображениями, чтобы библиотека могла найти её по ожидаемому пути
!ln -s /content/data /content/drive/MyDrive/Colab\ Notebooks/CNN_Sergienko_2025/siaug-main/data/ChestXRay-NIH

In [7]:
# Показывает список файлов и папок внутри директории проекта `siaug-main/data/`
!ls -l '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main/data/'

total 1
lrw------- 1 root root 13 May 17 10:41 ChestXRay-NIH -> /content/data


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # 1. Загрузка CSV
# df = pd.read_csv('/content/data/Data_Entry_2017_v2020.csv')

# # 2. Создание бинарной метки
# df['binary_label'] = df['Finding Labels'].apply(lambda x: 0 if x == 'No Finding' else 1)

# # 3. Деление на train/test (stratify сохраняет пропорции классов)
# train_df, test_df = train_test_split(
#     df,
#     test_size=0.2,
#     stratify=df['binary_label'],
#     random_state=42
# )

# # 4. Сохраняем
# train_df.to_csv('/content/data/train.csv', index=False)
# test_df.to_csv('/content/data/test.csv', index=False)

# print("Готово!")
# print(f"Train: {len(train_df)} записей")
# print(f"Test:  {len(test_df)} записей")

In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Загрузка оригинального CSV
df = pd.read_csv('/content/data/Data_Entry_2017_v2020.csv')

# 2. Создание бинарной метки: "No Finding" → 0, остальное → 1
df['binary_label'] = df['Finding Labels'].apply(lambda x: 0 if x == 'No Finding' else 1)

# 3. Список реально доступных изображений
available_images = set(os.listdir('/content/data/images'))

# 4. Оставляем только те строки, у которых файл реально существует
df = df[df['Image Index'].isin(available_images)]

# 5. Делим на train/test с сохранением пропорции классов
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['binary_label'],
    random_state=42
)

# 6. Сохраняем
train_df.to_csv('/content/data/train.csv', index=False)
test_df.to_csv('/content/data/test.csv', index=False)

print("✅ Готово!")
print(f"Train: {len(train_df)} записей")
print(f"Test:  {len(test_df)} записей")

✅ Готово!
Train: 27999 записей
Test:  7000 записей


In [9]:
import os

image_dir = "/content/data/images"
images = [f for f in os.listdir(image_dir) if f.endswith(".png") or f.endswith(".jpg")]
print("Количество изображений:", len(images))
print("Пример изображений:", images[:5])

Количество изображений: 34999
Пример изображений: ['00002184_001.png', '00003361_011.png', '00004858_078.png', '00003423_021.png', '00001040_002.png']


In [10]:
# Полностью удаляем старую Hydra
!pip uninstall -y hydra-core

# Ставим заново нужную нам версию (работает с Python 3.11)
!pip install hydra-core==1.3.2

Found existing installation: hydra-core 1.2.0
Uninstalling hydra-core-1.2.0:
  Successfully uninstalled hydra-core-1.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main
# !python siaug/train_repr.py experiment=nih_repr

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main
# HYDRA_FULL_ERROR=1 PYTHONPATH=. python siaug/train_repr.py experiment=nih_repr
# !PYTHONPATH=. python siaug/train_repr.py experiment=nih_repr



In [19]:
import os
os.makedirs("/content/outputs/logs", exist_ok=True)

In [23]:
%cd /content/drive/MyDrive/Colab\ Notebooks/CNN_Sergienko_2025/siaug-main
!PYTHONPATH=. python siaug/train_repr.py experiment=nih_repr

/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main
=> Starting [experiment=None]
=> Initializing Hydra configuration
Error executing job with overrides: ['experiment=nih_repr']
Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main/siaug/train_repr.py", line 27, in main
    cfg = instantiate(cfg)
          ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hydra/_internal/instantiate/_instantiate2.py", line 226, in instantiate
    return instantiate_node(
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hydra/_internal/instantiate/_instantiate2.py", line 366, in instantiate_node
    cfg[key] = instantiate_node(
               ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hydra/_internal/instantiate/_instantiate2.py", line 316, in instantiate_node
    items = [
            ^
  File "/usr/local/lib/python3.11/dist-packages/hydra/_internal/instantiate/_i

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/CNN_Sergienko_2025/siaug-main
!PYTHONPATH=. python siaug/train_repr.py --config-name=nih_repr


/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main
=> Starting [experiment=nih_repr]
=> Initializing Hydra configuration
[2025-05-18 07:02:12,894][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
2025-05-18 07:02:13.947248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747551733.974515   56474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747551733.981773   56474 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 07:02:14.007098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instr

In [ ]:
import os

path = '/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main/configs/criterion'
print(path)

for root, dirs, files in os.walk(path):
    level = root.replace(path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")


/content/drive/MyDrive/Colab Notebooks/CNN_Sergienko_2025/siaug-main/configs/criterion
criterion/
    tripod.yaml
    convirt.yaml
    simsiam.yaml
    bce.yaml
    xe.yaml
    default.yaml
    simsiam_default.yaml


In [ ]:
!ls siaug/dataloaders/nih_dataset.py

ls: cannot access 'siaug/dataloaders/nih_dataset.py': No such file or directory


In [ ]:
!pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
!find /content/data/images -name 00015700_010.png


In [ ]:
!PYTHONPATH=. HYDRA_FULL_ERROR=1 python siaug/train_repr.py --config-name=nih_repr

=> Starting [experiment=nih_repr]
=> Initializing Hydra configuration
[2025-05-18 06:00:10,794][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
2025-05-18 06:00:12.093457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747548012.162908   41282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747548012.180021   41282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 06:00:12.237201: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the a